# Oil and Gas Visualization/Dashboard

### Import required libraries

In [1]:
import datetime
import pymongo
from controls2 import *

client = pymongo.MongoClient()
database = client.anime
collection = database.myanimelist
options=init_components(collection)

In [2]:
def anime_recommandation(titles,collection,options,champs=["genres","duration","episodes","aired.start","main_title"],max_result=100):
    """
    Recommandation d'animés à partir d'autres d'animés

    Args:
        titles: titres des animés utilisé comme références
        collection: base de données comportant tous les animés
        champs: liste de champs utilisés pour la recommandation 
        max_result: taille maximale de la liste de recommandation
    
    Returns:
        liste d'animés recommandés
    """
    # Récupération d'infos sur les animés de références
    ref_infos=find_references(titles,collection,champs)

    # Recherche d'animés partageant possédeant les genres de référence
    under_max,selection=False,[]
    clean_yet=False
    if "genres" in ref_infos.keys():
        genres_c=[]
        for elt in ref_infos["genres"]:
            genres_c= genres_c + elt
        request=make_request(options=options,genres={"radio":'Limit to',"check":list(set(genres_c))})
        select=select_anime(request,collection,champs=["main_title"],sort={"fields":["score","popularity"],"order":[-1,-1]})
        select= set(select).difference(ref_infos["main_title"])
        selection.append(clean_selection(list(select),collection))
        clean_yet=True
        under_max=(len(selection[-1])<=max_result & len(selection[-1])>0)
        if not under_max:
            genres_c=[set(elt) for elt in ref_infos["genres"]]
            common_genres=genres_c[0]
            for elt in genres_c:
                common_genres = elt&common_genres
            if len(common_genres)==1:
                request=make_request(options=options,genres={'radio':'Exactly with',"check":list(common_genres)})
                select=select_anime(request,collection,champs=["main_title"],
                                    sort={"fields":["score","popularity"],"order":[-1,-1]})
                select= set(select).difference(ref_infos["main_title"])
                if len(select)>0:
                    selection.append(clean_selection(list(select),collection,clean_yet))
                    clean_yet=True
                    under_max=(len(selection[-1])<=max_result)
            elif len(common_genres)>1:
                common_genres=partiesliste(list(common_genres))
                len_common=-1
                selection2=[]
                Request=[]
                for elt in common_genres:
                    request=make_request(options=options,genres={'radio':'Exactly with',"check":elt})
                    if len_common!=len(elt):
                        Resquest.append([])
                    Request[-1].append(request)
                    len_common=len(elt)
                for r in Request:
                    select=select_anime({"$or":r},collection,champs["main_title"],
                                        sort={"fields":["score","popularity"],"order":[-1,-1]}))
                    selection2.append(clean_selection(select,collection,True))
                len_selection2=pd.Series(selection2).apply(lambda s: len(s) if len(s)!=0 else 10**30)
                best_selection2=len_selection2[len_selection2<=max_result]
                if len(best_selection2)>0:
                    selection.append(best_selection2[best_selection2.argmax()])
                else:
                    best_selection2=len_selection2[len_selection2>max_result]
                    selection.append(best_selection2[best_selection2.argmin()])
                under_max=(len(selection[-1])<=max_result & len(selection)>0)
                # for elt in common_genres:
                #     if not under_max:
                #         request=make_request(options=options,genres={'radio':'Exactly with',"check":elt})
                #         select=select_anime(request,collection,champs=["main_title"],
                #                             sort={"fields":["score","popularity"],"order":[-1,-1]})
                #         select= set(select).difference(ref_infos["main_title"])
                #         if common_genres.index(elt)==0:
                #             selection.append(clean_selection(list(select),collection))
                #         elif len(selection[-1])==0 or len(select)<len(selection[-1]):
                #             selection[-1]=clean_selection(list(select),collection)
                #             clean_yet=True
                #             under_max=(len(selection[-1])<=max_result)
                #     else:
                #         break

    # Recherche d'animés ayant environ la même durée
    if "duration" in ref_infos.keys() and not under_max:
        mean_duration=np.mean(ref_infos["duration"])
        limits=[round(0.75*mean_duration),round(1.25*mean_duration)]
        request=make_request(options=options,titles=selection[-1],duration={"check":[],"slider":limits})
        select=select_anime(request,collection,champs=["main_title"],sort={"fields":["score","popularity"],"order":[-1,-1]})
        if len(select)>0:
            selection.append(clean_selection(select,collection,clean_yet))
            clean_yet=True
            under_max=(len(selection[-1])<=max_result)

    # Recherche d'animés ayant environ le même nombre d'épisodes
    if "episodes" in ref_infos.keys() and not under_max:
        mean_episodes=np.mean(ref_infos["episodes"])
        limits=[round(0.75*mean_episodes),round(1.25*mean_episodes)]
        request=make_request(options=options,titles=selection[-1],episodes={"check":[],"slider":limits})
        select=select_anime(request,collection,champs=["main_title"],sort={"fields":["score","popularity"],"order":[-1,-1]})
        if len(select)>0:
            selection.append(clean_selection(select,collection,clean_yet))
            clean_yet=True
            under_max=(len(selection[-1])<=max_result)

    # Recherche d'animés ayant environ le même année
    if "aired" in ref_infos.keys() and not under_max:
        mean_year=np.mean(ref_infos["aired"])
        limits=[round(mean_year)-20,round(mean_year)+20]
        request=make_request(options=options,titles=selection[-1],year={"check":[],"slider":limits})
        select=select_anime(request,collection,champs=["main_title"],sort={"fields":["score","popularity"],"order":[-1,-1]})
        if len(select)>0:
            selection.append(clean_selection(select,collection,clean_yet))
            clean_yet=True
            under_max=(len(selection[-1])<=max_result)
    
    # Renvoi de la meilleur sélection
    len_selection=pd.Series(selection).apply(lambda s: len(s) if len(s)!=0 else 10**30)
   # print(len_selection)
    best_selection=selection[len_selection.argmin()]
    request=make_request(options=options,
    titles=clean_selection(best_selection,collection,clean_yet))
    best_selection= select_anime(request,collection,max_result=max_result,
                                    sort={"fields":["score","popularity"],"order":[-1,-1]})
    return best_selection

SyntaxError: invalid syntax (<ipython-input-2-dcb28b80a01a>, line 57)

In [3]:
anime_recommandation(["Naruto"],collection,options=options)

[{'_id': 6455,
  'main_title': 'Naruto: Shippuuden',
  'other_titles': ['Naruto Hurricane Chronicles',
   'ナルト- 疾風伝',
   'Naruto: Shippuden'],
  'score': 8.16,
  'synopsis': 'It has been two and a half years since Naruto Uzumaki left Konohagakure, the Hidden Leaf Village, for intense training following events which fueled his desire to be stronger. Now Akatsuki, the mysterious organization of elite rogue ninja, is closing in on their grand plan which may threaten the safety of the entire shinobi world. Although Naruto is older and sinister events loom on the horizon, he has changed little in personality—still rambunctious and childish—though he is now far more confident and possesses an even greater determination to protect his friends and home. Come whatever may, Naruto will carry on with the fight for what is important to him, even at the expense of his own body, in the continuation of the saga about the boy who wishes to become Hokage. [Written by MAL Rewrite]',
  'ranked': 349,
  '

In [62]:
import numpy as np
import pandas as pd
import plotly.plotly as py
import plotly.offline as pyo
import cufflinks as cf

### Import New York State dataset

In [63]:
df = pd.read_csv('data/wellspublic.csv', low_memory=False)
df.shape

(41716, 52)

In [64]:
df.columns

Index(['API_WellNo', 'Cnty', 'Hole', 'SideTrck', 'Completion', 'Well_Name',
       'Company_name', 'Operator_number', 'Well_Type', 'Map_Symbol',
       'Well_Status', 'Date_Status', 'Date_Permit_Application',
       'Permit_Issued', 'Date_Spudded', 'Date_Total_Depth',
       'Date_Well_Completed', 'Date_well_plugged', 'Date_well_confidential',
       'confid', 'town', 'quad', 'quadsec', 'Producing_name',
       'Producing_formation', 'Financial_security', 'Slant', 'County',
       'Region', 'State_lease', 'Proposed_depth', 'Surface_location',
       'Surface_Longitude', 'Surface_latitude', 'Bottom_hole_location',
       'Bottom_hole_longitude', 'Bottom_hole_latitude', 'True_vertical_depth',
       'Measured_depth', 'Kickoff', 'DrilledDepth', 'Elevation',
       'Original_well_type', 'Permit_Fee', 'Objective_formation', 'Depth_Fee',
       'Spacing', 'Spacing_Acres', 'Integration', 'Dt_Hearing', 'Dt_Mod',
       'LINK'],
      dtype='object')

### Make scattermapbox map

In [65]:
types = dict(
    BR = 'Brine',
    Confidential = 'Confidential',
    DH = 'Dry Hole',
    DS = 'Disposal',
    DW = 'Dry Wildcat',
    GD = 'Gas Development',
    GE = 'Gas Extension',
    GW = 'Gas Wildcat',
    IG = 'Gas Injection Well',
    IW = 'Enhanced Oil Recovery - Injection',
    LP = 'Liquefied Petroleum Gas Storage',
    MB = 'Monitoring Brine',
    MM = 'Monitoring Miscellaneous',
    MS = 'Monitoring Storage',
    NL = 'Not Listed',
    OB = 'Observation Well',
    OD = 'Oil Development',
    OE = 'Oil Extension',
    OW = 'Oil Wildcat',
    SG = 'Stratigraphic',
    ST = 'Storage',
    TH = 'Geothermal',
    UN = 'Unknown',
)

traces = []
for well, df in df.groupby('Well_Type'):
    trace = dict(
        type = 'scattermapbox',
        lon = df['Surface_Longitude'],
        lat = df['Surface_latitude'],
        text = df['Well_Name'],
        name = types[well],
        marker = dict(
            size = 4,
            opacity = 0.6,
        )
    )
    traces.append(trace)

# trace = dict(
#     type = 'scattermapbox',
#     lon = df['Surface_Longitude'],
#     lat = df['Surface_latitude'],
#     name = df['Well_Name'],
# )

# traces = [trace]

In [66]:
mapbox_access_token = 'pk.eyJ1IjoiamFja2x1byIsImEiOiJjaXhzYTB0bHcwOHNoMnFtOWZ3YWdreDB3In0.pjROwb9_CEuyKPE-x0lRUw'

layout = dict(

    title = "New York Oil and Gas map",

    # GENERAL LAYOUT
    width = 1280,
    height = 720,
    autosize = True,
    font = dict(
        family = "Overpass",
        size = 12,
        color = '#CCCCCC',
    ),
    margin = dict(
        t = 80,
        l = 40,
        b = 40,
        r = 120,
        pad = 0, 
    ),

    # OPTIONAL
    hovermode = "closest",
    
    # COLOR THEME
    plot_bgcolor = "#191A1A",
    paper_bgcolor = "#020202",
    
    # LEGEND
    legend = dict(
        x = 1.02,
        y = 1,
        font = dict(size = 10),
    ),

    # MAPBOX
    mapbox = dict(
        accesstoken = mapbox_access_token,
        style = "dark",
        center = dict(
            lon = -76.40,
            lat = 42.70,
        ),
        zoom = 5.5,
    ),

)

In [68]:
figure = dict(data=traces, layout=layout)
py.plot(figure, filename='Oil map')

'https://plot.ly/~jackluo/3594'